In [ ]:
import numpy as np
import pandas as pd
import datetime as dt 
import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
df_ppd = pd.read_csv("Annual_Crime_Dataset_2015.csv")
df_ppd.columns = ['GO Location', 'GO Report Date','GO Location Zip', 'GO Highest Offense Desc','GO X Coordinate','GO Y Coordinate']

NameError: name 'pd' is not defined

In [ ]:
df_crimeCount = df_ppd.count(['GO Highest Offence Desc'])

# Give meaningful names to the columns
df_crimeCount.columns = ['Zip Code', 'Crime_Count']

In [ ]:
df_notSafe = df_crimeCount.query("(Crime_Count  >= 10) & (Crime_Count <= 20)")

In [ ]:
df_notSafe

In [ ]:
address = 'Austin, TX'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Austin are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_Austin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_pref_loc['Latitude'], df_pref_loc['Longitude'], df_pref_loc['Crime_Count'], df_pref_loc['GO Location']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_austin

In [ ]:
CLIENT_ID = 'CWNI4CT0DGUI5O0HYDRMIJVVBZ4ZMEXBXNT2PDJ4IAG3ARF1' # your Foursquare ID
CLIENT_SECRET = 'EZ1ZNCFY0YDNE1SUMWB2UZTRAKR500VMRIAZHOTNKB5KOKHY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
location_venues = getNearbyVenues(names=df_pref_loc['GO Location'],
                                   latitudes=df_pref_loc['Latitude'],
                                   longitudes=df_pref_loc['Longitude']
                                  )

In [ ]:
location_venues


In [ ]:
location_venues.groupby('GO Location').count()


# Thank You